In [43]:
from pvlib.location import Location
from dotenv import load_dotenv
import googlemaps
from googlemaps.client import Client

import pandas as pd
import os





In [44]:
# google map API adaptor
from pydantic import BaseModel
from googlemaps.elevation import elevation
from googlemaps.timezone import timezone
from googlemaps.exceptions import ApiError, HTTPError, Timeout, TransportError
from datetime import datetime
from zoneinfo import ZoneInfo

class GoogleTimeZone(BaseModel):
    dstOffset: int | None
    rawOffset: int | None
    status: str
    timeZoneId: str | None
    timeZoneName: str | None
    def __init__(self, **kwarg):
        super().__init__(**kwarg)


class GoogleMap_Adaptor:
    def __init__(self, client: Client) -> None:
        self.__client = client
    
    def get_timezone(self, latitude: float, longitude: float) -> GoogleTimeZone|None:
        try:
            result = timezone(client=self.__client, location=(latitude, longitude),timestamp=datetime.now(tz=ZoneInfo('UTC')))
            return GoogleTimeZone(**result)
        except (ApiError, HTTPError, Timeout, TransportError) as e:
            print(f"Error fetching timezone: {str(e)}")
            return None
    
    def get_altitude(self, latitude: float, longitude: float) -> float | None:
        try:
            result = elevation(client=self.__client, locations=(latitude, longitude))
            return result[0]['elevation']
        except (ApiError, HTTPError, Timeout, TransportError) as e:
            print(f"Error fetching altitude: {str(e)}")
            return None
        except (IndexError, KeyError) as e:
            print(f"Error processing altitude data: {str(e)}")
            return None

In [45]:
# open weather API adaptor
from pydantic import BaseModel
from enum import Enum
import requests
class OpenWeather_General(BaseModel):
    temp:float|None
    feels_like:float|None
    temp_min:float|None
    temp_max:float|None
    pressure:float|None
    humidity:float |None
    sea_level:float|None
    grnd_level:float|None
    
    def __init__(self,**kwarg):
        super().__init__(**kwarg)

class OpenWeather_Wind(BaseModel):
    speed:float|None
    deg:float|None
    gust:float|None
    def __init__(self,**kwarg):
        super().__init__(**kwarg)

class OpenWeather_Unit(Enum):
    STANDARD='standard'
    METRIC='metric'
    IMPERIAL='imperial'

class OpenWeather_Adaptor:

    def __init__(self,apikey:str,unit:OpenWeather_Unit):
        self.__apikey=apikey
        self.unit=unit
    
    def get_currentWeather(self, latitude: float, longitude: float) -> tuple[OpenWeather_General, OpenWeather_Wind]:
        try:
            res = requests.get("https://api.openweathermap.org/data/2.5/weather", params={
                "lat": latitude,
                "lon": longitude,
                "appid": self.__apikey,
                "units": self.unit.value
            })
            res.raise_for_status()  # Raise an exception for bad status codes
            result = res.json()
            general_data = OpenWeather_General(**result['main'])
            wind_data = OpenWeather_Wind(**result["wind"])
            return general_data, wind_data
        except requests.RequestException as e:
            print(f"Error fetching weather data: {e}")
            return None, None
        except KeyError as e:
            print(f"Error parsing weather data: {e}")
            return None, None
        except Exception as e:
            print(f"Unexpected error: {e}")
            return None, None


In [46]:
# Solcast API adaptor
from datetime import datetime
from zoneinfo import ZoneInfo
from pydantic import BaseModel
from pandas import Timestamp,Timedelta

class Solcast_Irradiance(BaseModel):
    ghi:float
    ebh:float
    dni:float
    dhi:float
    cloud_opacity:float
    # period_end:Timestamp|None
    # diff_from_now:Timedelta|None
    period:str|None

    def __init__(self,**kwarg):
        super().__init__(**kwarg)
    
    def to_dict_irradiance(self)->dict:
        result={
            "ghi":self.ghi,
            "ebh":self.ebh,
            "dni":self.dni,
            "dhi":self.dhi,
            "cloud_opacity":self.cloud_opacity
        }

        return result




class Solcast_Adaptor:
    def __init__(self,apikey:str):
        self.__apikey=apikey

    def get_estimated_actuals(self,latitude:float,longitude:float)->pd.DataFrame|None:
        try:
            response=requests.get(f"https://api.solcast.com.au/world_radiation/estimated_actuals",
                                  params={
                                      "api_key":self.__apikey,
                                      "latitude": latitude,
                                      "longitude":longitude,
                                      "hours":1,
                                      "format":'json'
                                  })
            response.raise_for_status()
            result=response.json()
            return pd.DataFrame(data=result["estimated_actuals"])
        except requests.RequestException as e:
            print(f"Error fetching the irradiance data: {e}")
            return None
        except KeyError as e:
            print(f"Error parsing the irradiance data: {e}")
            return None
        except Exception as e:
            print(f"unexpected exception in `get_estimated_actual`:{e}")
            return None
        
    def get_current_irradiance(self,latitude:float,longitude:float)->Solcast_Irradiance|None:
        irradiance_estimated=self.get_estimated_actuals(latitude=latitude,longitude=longitude)
        if irradiance_estimated is None:
            print(f"could not load irradiance data from `get_estimate_actuals`")
            return None
        irradiance_estimated['period_end']=pd.to_datetime(irradiance_estimated['period_end'])
        now=datetime.now(tz=ZoneInfo('UTC'))
        irradiance_estimated["diff_from_now"]=abs(irradiance_estimated['period_end'] - now)
        index_min=irradiance_estimated['diff_from_now'].idxmin()
        result_irradiance=irradiance_estimated.loc[index_min]
        parameters=result_irradiance.to_dict()
        result=Solcast_Irradiance(**parameters)
        return result

        
    
        



In [47]:
# Solar farm site
from pandas import DataFrame
class SolarFarmLocation(Location):
    def __init__(self,latitude:float,longitude:float ,gmap_adaptor:GoogleMap_Adaptor,name:str, solcast_adaptor:Solcast_Adaptor,weather_adaptor:OpenWeather_Adaptor):
        tzInfo=gmap_adaptor.get_timezone(latitude=latitude,longitude=longitude)
        altitude=gmap_adaptor.get_altitude(latitude=latitude,longitude=longitude)
        # should raise exception in production
        if tzInfo is None:
            print("could not load time zone information")
        if altitude is None:
            print("could not load altitude")
        super().__init__(latitude, longitude, tz=tzInfo.timeZoneId, altitude=altitude, name=name)
        self.tzInfo=tzInfo
        # these two adaptor are used to get real-tiem data
        self.__solcast=solcast_adaptor
        self.__openWeather=weather_adaptor

    def get_current_weather(self)->DataFrame|None:
        result={}
        general,wind=self.__openWeather.get_currentWeather(latitude=self.latitude,longitude=self.longitude)
        irradiance=self.__solcast.get_current_irradiance(latitude=self.latitude,longitude=self.longitude)
        if irradiance is None:
            print(f"Not able to get irradiance data")
            return None
        result.update(irradiance.to_dict_irradiance())
        if wind is None:
            print(f"not able to get wind data")
            return None
        if general is None:
            print(f"not able to get temperature data in general")
        result.update({
            'temp_air':general.temp,
            'wind_speed':wind.speed
        })

        return DataFrame(data=[result],index=[pd.Timestamp.now(self.tz)])
        
        
    

    



In [48]:
# initialize google client
load_dotenv()
google_apikey=os.getenv("google_api_key")
openWeather_apikey=os.getenv("open_weather_api")
solcast_apikey=os.getenv('solcast_apikey')
googlemap_client=googlemaps.Client(key=google_apikey)
google_adaptor=GoogleMap_Adaptor(client=googlemap_client)
openweather_adaptor=OpenWeather_Adaptor(apikey=openWeather_apikey,unit=OpenWeather_Unit.METRIC)
solcast_adaptor=Solcast_Adaptor(apikey=solcast_apikey)

In [52]:

solar_farm=SolarFarmLocation(latitude=51.1327341457832,longitude=-114.15518620988793,name="Calgary",
                             gmap_adaptor=google_adaptor,
                             weather_adaptor=openweather_adaptor,
                             solcast_adaptor=solcast_adaptor)

solar_farm




Location: 
  name: Calgary
  latitude: 51.1327341457832
  longitude: -114.15518620988793
  altitude: 1240.6318359375
  tz: America/Edmonton

In [53]:
solar_farm.get_current_weather()

,ghi,ebh,dni,dhi,cloud_opacity,temp_air,wind_speed
2024-09-29 22:00:57.109401-06:00,0.0,0.0,0.0,0.0,24.0,4.74,9.26
